<a href="https://colab.research.google.com/github/mbatley/apachebeam/blob/main/side_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 87.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 30.8 MB

In [4]:
import apache_beam as beam
p1 = beam.Pipeline()

side_list = list()
with open ('customers_exclude.txt','r') as exclude_file:
  for cust_id in exclude_file:
    side_list.append(cust_id.rstrip())

print(side_list)

class SplitRow(beam.DoFn):
  def process(self,element,side_list):
    customer = element.split(',')
    if customer[0] not in side_list:
      return [customer]

class ProcessCustomers(beam.DoFn):
  def process(self,element,country,start_char):
    if(element[2]=='NY'):
      yield  element
    else:
      yield  beam.pvalue.TaggedOutput('Other_Cust',element)
    if(element[1].startswith('J')):
       yield  beam.pvalue.TaggedOutput('Names_J',element)



customers = (
    p1
    |beam.io.ReadFromText('Customers_age.txt')
    |beam.ParDo(SplitRow(),side_list)
    |beam.ParDo(ProcessCustomers(),country='NY',start_char='J').with_outputs('Names_J','Other_Cust',main='NewYork_Cust')
)

newyork_customers = customers.NewYork_Cust
other_customers = customers.Other_Cust
customer_withname_J = customers.Names_J

newyork_customers | 'Write Newyork Customers PCollection' >> beam.io.WriteToText("newyork")
other_customers  | 'Write Customers PCollection that lives in other cities' >> beam.io.WriteToText("customers_other_cities")
customer_withname_J  | 'Write Customers names with J PCollection' >> beam.io.WriteToText("customers_names_j")


p1.run()

['3', '7', '10']


In [5]:
! cat newyork-00000-of-00001

['1', 'John', 'NY', '22']
['6', 'Maria', 'NY', '15']
['9', 'Mona', 'NY', '18']


In [6]:
!cat customers_names_j-00000-of-00001

['1', 'John', 'NY', '22']
['2', 'Jim', 'LA', '25']


In [7]:
!cat customers_other_cities-00000-of-00001

['2', 'Jim', 'LA', '25']
['4', 'Albert', 'LA', '20']
['5', 'Samza.NY', '18']
['8', 'Kavita', 'LA', '20']
